In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from sklearn.tree import _tree
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# Load data
dataset_name = 'sampled_data2.csv'
df = pd.read_csv(dataset_name)
print(df.shape)
df.head()

In [ ]:
entity_column = 'customer'
target_column= 'fraud'
datetime_columns = 'step'

In [ ]:
if entity_column == '':
    df['entity_column']= df.index.tolist()
    entity_column = 'entity_column' 
    
if datetime_columns == '':
    df['datetime_columns']= df.index.tolist()    
    datetime_columns = 'datetime_columns'

numerical_columns = df._get_numeric_data().columns.values.tolist()
print(numerical_columns)
    

In [ ]:
X = df.drop(columns=[entity_column,target_column,datetime_columns])
y = df[target_column]

# Label Encoding of Categorical Columns
categorical_names = {}
for feature in X.columns:
    X[feature] =  X[feature].fillna('') 
    le =  LabelEncoder()
    le.fit(X[feature])
    X[feature] = le.transform(X[feature])
    categorical_names[feature] = le.classes_
    

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42,stratify=y)

rf = RandomForestClassifier(criterion='entropy', class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

predictions = rf.predict_proba(X_test)
predictions_class = rf.predict(X_test)


if df[target_column].nunique()> 2:
    rules_score = roc_auc_score(y_test, predictions[:,1], multi_class = 'ovo')
else:
    rules_score = roc_auc_score(y_test, predictions[:,1])

print(rules_score)

from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, predictions_class)
print(cf_matrix)

import seaborn as sns
sns.heatmap(cf_matrix, annot=True)

In [ ]:
'''
ROC_AUC = 0.9960853703703704

Confusion Matrix = 
[[13193   307]
 [  243  5157]]
'''